In [1]:
# import necessary libraries
import numpy as np
import pandas as pd

In [2]:
# Use these values to calculate scores
gap_penalty = -2
match_award = 1
mismatch_penalty = -1

In [3]:
def Needleman_Wunsch_global_alignment(s1,s2):
    m = len(s1)
    n = len(s2)
    ga = np.zeros((n+1,m+1))
    w2 = ['-'] + list(s2)
    w1 = ['/'] + list(s1)
    for i in range(1,len(w1)):
        ga[0,i] = ga[0,i-1]-2
    for j in range(1,len(w2)):
        ga[j,0] = ga[j-1,0]-2
    for i in range(1,len(w1)):
        for j in range(1,len(w2)):
            if(w1[i]==w2[j]):
                ga[j,i] = max(ga[j-1,i] + gap_penalty,ga[j,i-1] + gap_penalty,ga[j-1,i-1] + match_award)
            elif(w1[i]=='-' or w2[j]=='-'):
                ga[j,i] = max(ga[j-1,i] + gap_penalty,ga[j,i-1] + gap_penalty,ga[j-1,i-1] + gap_penalty)
            else:
                ga[j,i] = max(ga[j-1,i] + gap_penalty,ga[j,i-1] + gap_penalty,ga[j-1,i-1] + mismatch_penalty)

            
            
    return(ga)
                
                
                
    

In [4]:
s1 = "CGCTCGCT"
s2 = "CACTCGT"

g = Needleman_Wunsch_global_alignment(s1,s2)
w2 = ['-'] + list(s2)
w1 = ['/'] + list(s1)
ga = pd.DataFrame(g, index = w2,columns = w1)
ga


,/,C,G,C,T,C,G,C,T
-,0.0,-2.0,-4.0,-6.0,-8.0,-10.0,-12.0,-14.0,-16.0
C,-2.0,1.0,-1.0,-3.0,-5.0,-7.0,-9.0,-11.0,-13.0
A,-4.0,-1.0,0.0,-2.0,-4.0,-6.0,-8.0,-10.0,-12.0
C,-6.0,-3.0,-2.0,1.0,-1.0,-3.0,-5.0,-7.0,-9.0
T,-8.0,-5.0,-4.0,-1.0,2.0,0.0,-2.0,-4.0,-6.0
C,-10.0,-7.0,-6.0,-3.0,0.0,3.0,1.0,-1.0,-3.0
G,-12.0,-9.0,-6.0,-5.0,-2.0,1.0,4.0,2.0,0.0
T,-14.0,-11.0,-8.0,-7.0,-4.0,-1.0,2.0,3.0,3.0


In [17]:
def match_score(x,y):
    if(x==y):
        return(match_award)
    elif(x=='-' or y=='-'):
        return(gap_penalty)
    else:
        return(mismatch_penalty)
        

In [18]:
def traceback(seq1,seq2):
    
    score = Needleman_Wunsch_global_alignment(seq1,seq2)
    # Create variables to store alignment
    align1 = ""
    align2 = ""
    
    m = len(seq2)
    n = len(seq1)
    
    # Start from the bottom right cell in matrix
    i = m
    j = n

    # We'll use i and j to keep track of where we are in the matrix, just like above
    while i > 0 and j > 0: # end touching the top or the left edge
        score_current = score[i][j]
        score_diagonal = score[i-1][j-1]
        score_up = score[i][j-1]
        score_left = score[i-1][j]
        
        # Check to figure out which cell the current score was calculated from,
        # then update i and j to correspond to that cell.
        if(score_current == score_diagonal + match_score(seq1[j-1], seq2[i-1])):
            align1 += seq1[j-1]
            align2 += seq2[i-1]
            i -= 1
            j -= 1
        elif(score_current == score_up + gap_penalty):
            align1 += seq1[j-1]
            align2 += '-'
            j -= 1
        elif(score_current == score_left + gap_penalty):
            align1 += '-'
            align2 += seq2[i-1]
            i -= 1
    # Finish tracing up to the top left cell
    while j > 0:
        align1 += seq1[j-1]
        align2 += '-'
        j -= 1
    while i > 0:
        align1 += '-'
        align2 += seq2[i-1]
        i -= 1
    
    # Since we traversed the score matrix from the bottom right, our two sequences will be reversed.
    # These two lines reverse the order of the characters in each sequence.
    align1 = align1[::-1]
    align2 = align2[::-1]
    
    return(align1, align2)



In [19]:
Needleman_Wunsch_global_alignment(s11,s12)

array([[  0.,  -2.,  -4.,  -6.,  -8., -10., -12.],
       [ -2.,   1.,  -1.,  -3.,  -5.,  -7.,  -9.],
       [ -4.,  -1.,   0.,   0.,  -2.,  -4.,  -6.],
       [ -6.,  -3.,  -2.,  -1.,   1.,  -1.,  -3.],
       [ -8.,  -5.,  -2.,  -3.,  -1.,   0.,   0.],
       [-10.,  -7.,  -4.,  -3.,  -3.,   0.,  -1.]])

In [20]:
s11 = "ttgact"
s12 = "ttgatc"
s21="gttcac"
s22="attcgt"
s31="ctcaggt"
s32="ctgagct"

output1, output2 = traceback(s11.upper(), s12.upper())

print(output1 + "\n" + output2)    
    

TTGACT
TTGATC
